# One Tree Planted
https://onetreeplanted.org

In [13]:
#climate impact data is explorer instead
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests

In [14]:
# with open('../input/OneTreePlaneted/OneTreePlanted.json', 'r') as file:
#     data = json.load(file)
# type(data)
base_url = 'https://services8.arcgis.com/pKd0yHtOd6niWQzj/ArcGIS/rest/services/ActiveProjects11June24_XY/FeatureServer/436/query'

#Defining Parameters
params = {
    'f': 'json',
    'returnIdsOnly': 'false',
    'returnCountOnly': 'false',
    'orderByFields': '',
    'outSR': '102100',
    'returnGeometry': 'true',
    'spatialRel': 'esriSpatialRelIntersects',
    'where': "Project_Status IN ('Complete','Planted/Reporting','OTP Reporting','Monitoring & Maintenance')"
}

response = requests.get(base_url, params=params)


if response.status_code == 200:
    data = response.json()
    
    # Extracting the  features
    features = data.get('features', [])
    
    extracted_data = []
    
    for feature in features:
        attributes = feature.get('attributes', {})
        geometry = feature.get('geometry', {})
        
        # Handling null values and missing fields
        row = {
            'Project_Title': attributes.get('Project_Title', None),
            'Longitude': geometry.get('x', None),
            'Latitude': geometry.get('y', None)
        }
        extracted_data.append(row)
    

    df = pd.DataFrame(extracted_data)
    
  
    

In [15]:
if isinstance(data, list) and isinstance(data[0], dict):
    df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 819 entries, 0 to 818
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Project_Title  819 non-null    object 
 1   Longitude      812 non-null    float64
 2   Latitude       812 non-null    float64
dtypes: float64(2), object(1)
memory usage: 19.3+ KB


### Fix geometries

In [16]:
df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

In [17]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs = 'EPSG:3857')
gdf['geometry'] = gdf['geometry'].make_valid()

In [18]:
gdf.head(1)

,Project_Title,Longitude,Latitude,geometry
0,Ireland 2022 - Nationwide Community Planting,-840006.413494,6.999668e+06,POINT (-840006.413 6999667.501)


### Harmonize nomenclature

In [19]:
gdf = gdf[['Project_Title', 'geometry']]

columns_rename_mapping = {
    'Project_Title': 'project_description_reported'
}
gdf.rename(columns=columns_rename_mapping, inplace=True)

In [20]:
invalid_geom = False

if len(gdf[gdf.geometry.is_valid == False]) > 0:
    invalid_geom = True

In [21]:
gdf['url'] = 'https://onetreeplanted.org'
gdf['host_name'] = 'One Tree Planted'
gdf['project_geometries_invalid'] = invalid_geom
gdf.reset_index(inplace = True, names = 'project_id_reported')
gdf = gdf.assign(project_description_reported=None,site_sqkm=None,species_count_reported=None, species_planted_reported=None, survival_rate_reported=None,trees_planted_reported=None,planting_date_reported=None,country=None,site_id_reported=None)
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 819 entries, 0 to 818
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   project_id_reported           819 non-null    int64   
 1   project_description_reported  0 non-null      object  
 2   geometry                      819 non-null    geometry
 3   url                           819 non-null    object  
 4   host_name                     819 non-null    object  
 5   project_geometries_invalid    819 non-null    bool    
 6   site_sqkm                     0 non-null      object  
 7   species_count_reported        0 non-null      object  
 8   species_planted_reported      0 non-null      object  
 9   survival_rate_reported        0 non-null      object  
 10  trees_planted_reported        0 non-null      object  
 11  planting_date_reported        0 non-null      object  
 12  country                       0 non-null  

In [22]:
gdf["planting_date_type"]= " "

In [23]:
gdf.to_crs('EPSG:4326').to_file('../midsave/one_tree_planted.gpkg')